In [2]:
import pandas as pd #https://www.kdnuggets.com/2020/07/spam-filter-python-naive-bayes-scratch.html

## DATA SETS ---------
smsData = pd.read_csv("spam.csv", encoding = "latin-1", usecols=["v1", "v2"])## importa o data set como um data-frame

## TRANSFORMS
smsData.columns = ["Status", "Mensagem"]

print(smsData.shape)
smsData.head()

(5572, 2)


,Status,Mensagem
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
smsData['Status'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Status, dtype: float64

In [5]:
# Randomize the dataset
data_randomized = smsData.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Split into training and test sets
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [6]:
training_set['Status'].value_counts(normalize=True)

ham     0.864065
spam    0.135935
Name: Status, dtype: float64

In [7]:
test_set['Status'].value_counts(normalize=True)

ham     0.873429
spam    0.126571
Name: Status, dtype: float64

In [8]:
# Before cleaning
training_set.head(3)

,Status,Mensagem
0,ham,Convey my regards to him
1,ham,"[Û_] anyway, many good evenings to u! s"
2,ham,My sort code is and acc no is . The bank is n...


In [9]:
# After cleaning
training_set['Mensagem'] = training_set['Mensagem'].str.replace(
   '\W', ' ') # Removes punctuation
training_set['Mensagem'] = training_set['Mensagem'].str.lower()
training_set.head(3)

,Status,Mensagem
0,ham,convey my regards to him
1,ham,û_ anyway many good evenings to u s
2,ham,my sort code is and acc no is the bank is n...


In [10]:
training_set['Mensagem'] = training_set['Mensagem'].str.split()

vocabulary = []
for sms in training_set['Mensagem']:
   for word in sms:
      vocabulary.append(word)

vocabulary = list(set(vocabulary))

In [11]:
len(vocabulary)

7766

In [12]:
word_counts_per_sms = {'secret': [2,1,1],
                       'prize': [2,0,1],
                       'claim': [1,0,1],
                       'now': [1,0,1],
                       'coming': [0,1,0],
                       'to': [0,1,0],
                       'my': [0,1,0],
                       'party': [0,1,0],
                       'winner': [0,0,1]
                      }

word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,secret,prize,claim,now,coming,to,my,party,winner
0,2,2,1,1,0,0,0,0,0
1,1,0,0,0,1,1,1,1,0
2,1,1,1,1,0,0,0,0,1


In [14]:
word_counts_per_sms = {unique_word: [0] * len(training_set['Mensagem']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['Mensagem']):
   for word in sms:
      word_counts_per_sms[word][index] += 1

In [19]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,is,fav,grave,rest,gre,ten,only1more,bajarangabali,ki,sigh,...,block,want,ain,tulip,4txt,lacking,peach,crucial,toyota,idew
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Status,Mensagem,is,fav,grave,rest,gre,ten,only1more,bajarangabali,...,block,want,ain,tulip,4txt,lacking,peach,crucial,toyota,idew
0,ham,"[convey, my, regards, to, him]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[û_, anyway, many, good, evenings, to, u, s]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[my, sort, code, is, and, acc, no, is, the, ba...",3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[sorry, i, din, lock, my, keypad]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,spam,"[hi, babe, its, chloe, how, r, u, i, was, smas...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Status'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Status'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam
n_words_per_spam_message = spam_messages['Mensagem'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['Mensagem'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

In [22]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
   n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
   p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
   parameters_spam[word] = p_word_given_spam

   n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
   p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
   parameters_ham[word] = p_word_given_ham

In [23]:
import re

def classify(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham: 
         p_ham_given_message *= parameters_ham[word]

   print('P(Spam|message):', p_spam_given_message)
   print('P(Ham|message):', p_ham_given_message)

   if p_ham_given_message > p_spam_given_message:
      print('Label: Ham')
   elif p_ham_given_message < p_spam_given_message:
      print('Label: Spam')
   else:
      print('Equal proabilities, have a human classify this!')

In [24]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 7.551469020619691e-26
P(Ham|message): 3.13163390665657e-27
Label: Spam


In [25]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 4.914731817244035e-25
P(Ham|message): 4.052730830087535e-21
Label: Ham


In [26]:
def classify_test_set(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham:
         p_ham_given_message *= parameters_ham[word]

   if p_ham_given_message > p_spam_given_message:
      return 'ham'
   elif p_spam_given_message > p_ham_given_message:
      return 'spam'
   else:
      return 'needs human classification'

In [28]:
test_set['predicted'] = test_set['Mensagem'].apply(classify_test_set)
test_set.head()

,Status,Mensagem,predicted
0,ham,S...from the training manual it show there is ...,ham
1,spam,Do you want a new Video phone? 600 anytime any...,spam
2,ham,True. Its easier with her here.,ham
3,ham,Midnight at the earliest,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [30]:
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
   row = row[1]
   if row['Status'] == row['predicted']:
      correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1107
Incorrect: 7
Accuracy: 0.9937163375224417


In [31]:
classify("Chupamos valadão, ahhhhhhh caralho odeio esta cadeira")

P(Spam|message): 0.13593539703903096
P(Ham|message): 0.8640646029609691
Label: Ham
